In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install -q python-dotenv openai azure-identity 

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

print(os.getenv("AOAI_ENDPOINT"))
print(os.getenv("AOAI_KEY"))

client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    api_version="2023-05-15",
    api_key=os.getenv("AOAI_KEY")
    )

models = client.models.list()

with open('azure-oai-models.json', 'w') as file:
    models_dict = [model.__dict__ for model in models]
    json.dump(models_dict, file)

for model in models:
    print("ID:", model.id)
    print("Current status:", model.lifecycle_status)
    print("Model capabilities:", model.capabilities)
    print("-------------------")


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    api_version="2024-05-01-preview",
    api_key=os.getenv("AOAI_KEY"))

GPT_MODEL = "gpt-35-turbo"

prompt_startphrase = "Suggest three names for a new pet salon business. The generated name ideas should evoke positive emotions and the following key features: Professional, friendly, Personalized Service."

response = client.completions.create(
    model=GPT_MODEL,
    prompt=prompt_startphrase,
    temperature=0.7,
    max_tokens=100,
    #best_of=5,
    n=3,
    stop=None)

responsetext = response.choices[0].text

print("Prompt:" + prompt_startphrase + "\nResponse:" + responsetext)


In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    api_version="2024-05-01-preview",
    api_key=os.getenv("AOAI_KEY"))

# This model name is what you chose when you deployed the model in Azure OpenAI
GPT_MODEL = "gpt-35-turbo"

prompt_startphrase = "Suggest three names for a new pet salon business. The generated name ideas should evoke positive emotions and the following key features: Professional, friendly, Personalized Service."

response = client.completions.create(
    model=GPT_MODEL,  
    prompt=prompt_startphrase,
    temperature=0.8,
    max_tokens=100,
    logit_bias={
        30026:-100,
        81:-100,
        9330:-100,
        808:-100,
        42114:-100,
        1308:-100, 
        3808:-100,
        502:-100,
        322:-100}
    )  
  
responsetext = response.choices[0].text
print(responsetext)

In [46]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    api_version="2024-05-01-preview",
    api_key=os.getenv("AOAI_KEY"))

# This model name is what you chose when you deployed the model in Azure OpenAI
GPT_MODEL = "gpt-35-turbo"

response = client.chat.completions.create(
    model=GPT_MODEL,
    messages = [
        {"role":"system","content":"You are an AI assistant that helps people find information."},
        {"role":"user","content":"Hello world"},
        {"role":"assistant","content":"Hello! How can I assist you today?"},
        {"role":"user","content":"I want to know more about pets and why dogs are good for humans?"}],
    temperature=0.8,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

print(response.choices[0].message.content)

Sure, I can help you with that. 

Pets, especially dogs, can provide numerous benefits for humans. Here are a few reasons why dogs are good for humans:

1. Companionship: Dogs are loyal companions and can provide a great source of comfort and emotional support for their owners.

2. Exercise: Dogs need regular exercise, so they can motivate their owners to get outside and engage in physical activity, which is beneficial for both the owner and the dog.

3. Stress reduction: Studies have shown that spending time with dogs can reduce stress levels and promote relaxation.

4. Socialization: Dogs can also help their owners socialize and meet new people, as they often act as conversation starters and can help break the ice in social situations.

5. Health benefits: Some studies have suggested that owning a dog can lower blood pressure, reduce the risk of heart disease, and even boost the immune system.

Overall, dogs can bring a lot of joy and happiness to people's lives, and can be great com

In [ ]:
import os
from openai import AzureOpenAI
import tiktoken

client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    api_version="2024-05-01-preview",
    api_key=os.getenv("AOAI_KEY"))

# This model name is what you chose when you deployed the model in Azure OpenAI
GPT_MODEL = "gpt-35-turbo"

system_message = {"role": "system", "content": "You are a helpful assistant."}
max_response_tokens = 250
token_limit = 4096
conversation = []
conversation.append(system_message)

def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")
    num_tokens = 0
    for message in messages:
        num_tokens += 4           # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":     # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2               # every reply is primed with <im_start>assistant
    return num_tokens

print("I am a helpful assistant. I can talk about pets and salons. What would you like to talk about?")

while True:
    user_input = input("")     
    conversation.append({"role": "user", "content": user_input})
    conv_history_tokens = num_tokens_from_messages(conversation)

    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)

    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=conversation,
        temperature=0.8,
        max_tokens=max_response_tokens)

    conversation.append({"role": "assistant", "content": response.choices[0].message.content})
    print("\n" + response.choices[0].message.content)
    print("(Tokens used: " + str(response.usage.total_tokens)  + ")")

# Create Client with gpt-35-turbo model

In [51]:
from dotenv import load_dotenv
from openai import AzureOpenAI
import os

load_dotenv()

client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    api_version="2024-06-01",
    api_key=os.getenv("AOAI_KEY"))

#GPT_MODEL = "gpt-35-turbo"
GPT_MODEL = "gpt-4"

def chat_msg(prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])
    print(response.choices[0].message.content)

# Prompt Design

In [52]:
text_prompt = "how grammar works"
chat_msg(text_prompt)

Grammar is the structure and system of a language, consisting of syntax, morphology, and often phonetics, among other things. It establishes rules for composing clauses, phrases, and words in any language. Here's a basic breakdown of how grammar works in English:

1. **Parts of Speech**: Understanding parts of speech is fundamental in learning how grammar works. The main parts of speech include:
   - **Nouns**: Words used to identify people, places, things, or ideas (e.g., cat, school, happiness).
   - **Pronouns**: Words that take the place of nouns (e.g., he, she, it, they).
   - **Verbs**: Words that express action or being (e.g., run, seem).
   - **Adjectives**: Words that describe nouns (e.g., blue, quick, tall).
   - **Adverbs**: Words that modify verbs, adjectives, or other adverbs (e.g., quickly, very, well).
   - **Prepositions**: Words used before nouns or pronouns to form phrases indicating time, place, or possession (e.g., in, on, at, between).
   - **Conjunctions**: Words 

# Summarize Text - \n\nTl;dr

In [53]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"
chat_msg(prompt)

Scaling up language models significantly boosts their ability to perform new language tasks with minimal examples, sometimes matching or surpassing the performance of models that have been extensively trained on task-specific data.


# Classify Text

In [54]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
chat_msg(prompt)

Hardware Support


# Generate new product names

In [55]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."
chat_msg(prompt)

Product names: AdaptStep, OmniShoe, FitFlex, UniSize Step


# Ask Q & A

In [56]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""
chat_msg(prompt)

The 2020 Summer Olympics men's high jump ended in a rare and remarkable tie for the gold medal between Mutaz Essa Barshim of Qatar and Gianmarco Tamberi of Italy.


# Analyzing Customer Feedback

In [57]:
prompt = """Decide whether the following customer feedback is positive or negative.

Q: I was disappointed with the quality of the product. It was very cheaply made and did not meet my expectations at all.
Q: I was happy with this product, it is well made and great quality for the price.
"""
chat_msg(prompt)

The first customer feedback is negative, and the second customer feedback is positive.


# Extracting PII Data

In [58]:
prompt = """List all PII data from following statement:
John Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Microsft. He has a wife named Jane Doe and they have two children
"""
chat_msg(prompt)

Here is the Personally Identifiable Information (PII) found in the statement:

1. Full name: John Doe
2. Age: 35
3. Home address: 21 Main Street, New York, NY
4. Employer: Microsoft
5. Spouse's name: Jane Doe
6. Mention of having two children (indirectly personal information related to family structure)


# Embeddings
- find similarity between words, sentences, and documents

In [59]:
import numpy as np
model="text-embedding-3-small"

def cosine_similarity(query_embedding, embeddings, distance_metric='cosine'):
    if distance_metric == 'cosine':
        distances = np.dot(embeddings, query_embedding) / (np.linalg.norm(embeddings) * np.linalg.norm(query_embedding))
        distances = 1 - distances  
    else:
        raise ValueError("Métrica de distância não suportada. Utilize 'cosine'.")

    return distances

automobile_embedding    = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding       = client.embeddings.create(input='vehicle', model=model).data[0].embedding
car_embedding           = client.embeddings.create(input='car', model=model).data[0].embedding
dinosaur_embedding      = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding         = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, car_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

0.36813609201882325
0.44352696470486064
0.6689407064684705
0.7729866110986178


# Comparing article from news dataset

In [61]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"' 
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head() 

0.6693155375025988
0.8841862966109504


In [62]:

article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

0.6693155375025988
0.8841862966109504


# Code Generation

## Model Parameters
- temperature : Controls the randomness of the output. A value of 0 makes the output more deterministic
- max_tokens : The maximum number of tokens in the generated output.
- top_p : Controls diversity via nucleus sampling.
- frequency_penalty and presence_penalty : These parameters are set to 0 to avoid penalizing the generation of similar words.
- stop : Specifies stopping sequences to end the generated text appropriately.


In [63]:
from dotenv import load_dotenv
from openai import AzureOpenAI
import os

load_dotenv()

client = AzureOpenAI(
    azure_endpoint=os.getenv("AOAI_ENDPOINT"),
    api_version="2024-06-01",
    api_key=os.getenv("AOAI_KEY"))

#GPT_MODEL = "gpt-35-turbo"
GPT_MODEL = "gpt-4"

def gen_code(prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},],
        temperature=0,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["#",";"] )
    
    print(response.choices[0].message.content)


# Generate SQL Queries  

In [64]:
prompt = "### SQL Server tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)
gen_code(prompt)

### SQL Server tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months

 query: 
To create a SQL query that lists the names of departments which employed more than 10 employees in the last 3 months, we need to consider the `Employee` and `Department` tables, and also take into account the `Salary_Payments` table to filter employees active in the last 3 months based on their salary records. Here's how you can structure the query:

```sql
SELECT d.name
FROM Department d
JOIN Employee e ON d.id = e.department_id
JOIN Salary_Payments sp ON e.id = sp.employee_id
WHERE sp.date >= DATEADD(MONTH, -3, GETDATE())
GROUP BY d.name
HAVING COUNT(DISTINCT e.id) > 10


# Explaining SQL Code

In [65]:
code = "Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "

gen_code(code)

Certainly! The SQL query you provided is designed to retrieve the names of departments where more than 10 employees have received salary payments in the last 3 months. Here's a breakdown of each part of the query:

1. **SELECT d.name**: This part of the query specifies that the output will be the names of the departments.

2. **FROM Department d JOIN Employee e ON d.id = e.department_id**: This part joins two tables: `Department` and `Employee`. It joins these tables on the department ID, meaning it links each employee to their respective department.

3. **WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months')**: This condition filters the employees to include only those who have entries in the `Salary_Payments` table within the last 3 months. The `now() - interval '3 months'` part calculates the date that is three months before the current date.

4. **GROUP BY d.name**: This groups the results by the department name. Since the output needs to 